In [1]:
from pyterrier import started, init

if not started():
    init()

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
from fare.modules.topics_loader import parse_topics
from pandas import DataFrame

topics: DataFrame = parse_topics()
topics

,qid,query,description,narrative,object_first,object_second
0,2,"Which is better, a laptop or a desktop?",A user wants to buy a new PC but has no prior ...,Highly relevant documents will describe the ma...,laptop,desktop
1,3,"Which is better, Canon or Nikon?",A user wants to buy a new camera from one of t...,Highly relevant documents will compare pairs o...,Canon,Nikon
2,8,What are the advantages and disadvantages of P...,"With no previous coding experience, a young us...",Relevant documents may contain an overview of ...,PHP,Python
3,9,Why is Linux better than Windows?,"Being tired of uncontrolled updates, a Windows...",Given a specific intent to find information in...,Linux,Windows
4,12,Train or plane? Which is the better choice?,A frequent traveler (domestic and internationa...,A highly relevant document will contrast the e...,Train,plane
5,14,Should one prefer Chinese medicine or Western ...,Given the medical outcome of a recent visit to...,A highly relevant document will contrast the g...,Chinese medicine,Western medicine
6,17,"Do you prefer cats or dogs, and why?",A person thinks of getting a pet but is not su...,Highly relevant documents will discuss advanta...,cats,dogs
7,18,What is the better way to grill outdoors: gas ...,A person having just decided to start their ve...,Highly relevant documents contain pros and con...,gas,charcoal
8,19,"Which is better, MAC or PC?",A person is thinking about replacing their old...,Highly relevant documents discuss differences ...,MAC,PC
9,22,"Which is better, Pepsi or Coke?","When it comes to soft drinks, there are two ki...",Highly relevant documents contain a comparison...,Pepsi,Coke


In [7]:
from pathlib import Path

run_file_path = Path("data/runs/Captain-Levi/output/run4.txt")

In [8]:
from fare.modules.fairness_reranker import FairnessReranker
from fare.modules.stance_reranker import StanceReranker
from fare.modules.stance_tagger import StanceTagger
from fare.modules.text_loader import TextLoader
from fare.modules.runs_loader import RunLoader

pipeline = RunLoader(run_file_path)
pipeline = ~(pipeline >> TextLoader())
pipeline = ~(pipeline >> StanceTagger.GROUND_TRUTH)
pipeline_stance_first = ~(pipeline % 5 >> StanceReranker.STANCE_FIRST) ^ pipeline
pipeline_alternating_stance = ~(
        pipeline_stance_first % 5 >> FairnessReranker.ALTERNATING_STANCE) ^ pipeline_stance_first

In [19]:
results: DataFrame = (pipeline % 5).transform(topics)
results = results[["qid", "query", "docno", "rank", "stance_label"]]
results

,qid,query,docno,rank,stance_label
0,2,"Which is better, a laptop or a desktop?",clueweb12-0301wb-84-10147___2,0,FIRST
1,2,"Which is better, a laptop or a desktop?",clueweb12-0907wb-60-03588___2,1,SECOND
2,2,"Which is better, a laptop or a desktop?",clueweb12-0013wb-19-15392___6,2,SECOND
3,2,"Which is better, a laptop or a desktop?",clueweb12-1804wb-90-15876___2,3,NEUTRAL
4,2,"Which is better, a laptop or a desktop?",clueweb12-1201wb-97-16663___1,4,SECOND
...,...,...,...,...,...
49000,100,Should I learn Python or R for data analysis?,clueweb12-1509wb-05-28610___12,0,NEUTRAL
49001,100,Should I learn Python or R for data analysis?,clueweb12-1812wb-50-24680___33,1,NO
49002,100,Should I learn Python or R for data analysis?,clueweb12-0800wb-21-10162___3,2,NO
49003,100,Should I learn Python or R for data analysis?,clueweb12-1213wb-15-28891___15,3,NEUTRAL


In [20]:
results_stance_first: DataFrame = (pipeline_stance_first % 5).transform(topics)
results_stance_first = results_stance_first[["qid", "query", "docno", "rank", "stance_label"]]
results_stance_first

,qid,query,docno,rank,stance_label
245,100,Should I learn Python or R for data analysis?,clueweb12-1509wb-05-28610___12,0,NEUTRAL
246,100,Should I learn Python or R for data analysis?,clueweb12-1213wb-15-28891___15,1,NEUTRAL
247,100,Should I learn Python or R for data analysis?,clueweb12-0800wb-21-10162___4,2,FIRST
248,100,Should I learn Python or R for data analysis?,clueweb12-1812wb-50-24680___33,3,NO
249,100,Should I learn Python or R for data analysis?,clueweb12-0800wb-21-10162___3,4,NO
...,...,...,...,...,...
240,95,"Which is more environmentally friendly, a hybr...",clueweb12-0106wb-08-06338___9,0,SECOND
241,95,"Which is more environmentally friendly, a hybr...",clueweb12-1700tw-36-07362___4,1,NEUTRAL
242,95,"Which is more environmentally friendly, a hybr...",clueweb12-0902wb-51-01399___8,2,FIRST
243,95,"Which is more environmentally friendly, a hybr...",clueweb12-0508wb-75-09549___12,3,NEUTRAL


In [21]:
results_alternating_stance: DataFrame = (pipeline_alternating_stance % 5).transform(topics)
results_alternating_stance = results_alternating_stance[["qid", "query", "docno", "rank", "stance_label"]]
results_alternating_stance

,qid,query,docno,rank,stance_label
0,100,Should I learn Python or R for data analysis?,clueweb12-1509wb-05-28610___12,0,NEUTRAL
1,100,Should I learn Python or R for data analysis?,clueweb12-1213wb-15-28891___15,1,NEUTRAL
2,100,Should I learn Python or R for data analysis?,clueweb12-0800wb-21-10162___4,2,FIRST
3,100,Should I learn Python or R for data analysis?,clueweb12-1812wb-50-24680___33,3,NO
4,100,Should I learn Python or R for data analysis?,clueweb12-0800wb-21-10162___3,4,NO
...,...,...,...,...,...
245,95,"Which is more environmentally friendly, a hybr...",clueweb12-0106wb-08-06338___9,0,SECOND
246,95,"Which is more environmentally friendly, a hybr...",clueweb12-1700tw-36-07362___4,1,NEUTRAL
247,95,"Which is more environmentally friendly, a hybr...",clueweb12-0902wb-51-01399___8,2,FIRST
248,95,"Which is more environmentally friendly, a hybr...",clueweb12-0508wb-75-09549___12,3,NEUTRAL


In [39]:
from pandas import concat

qid = "19"


compare_results = results[results["qid"] == qid].reset_index()
doc_chars = {
    docno: char
    for docno, char in zip(compare_results["docno"], "ABCDEFGHIJKLMNOPQRSTUVWXYZ")
}
compare_results["docno"] = compare_results["docno"].apply(lambda docno: doc_chars[docno])
compare_results_stance_first = results_stance_first[results_stance_first["qid"] == qid].reset_index()
compare_results_stance_first["docno"] = compare_results_stance_first["docno"].apply(lambda docno: doc_chars[docno])
compare_results_alternating_stance = results_alternating_stance[results_alternating_stance["qid"] == qid].reset_index()
compare_results_alternating_stance["docno"] = compare_results_alternating_stance["docno"].apply(lambda docno: doc_chars[docno])
compare_results_merged: DataFrame = concat([
    # compare_results[["qid", "query", "rank"]],
    compare_results[["docno", "stance_label"]],
    compare_results_stance_first[["docno", "stance_label"]],
    compare_results_alternating_stance[["docno", "stance_label"]],
], axis=1)
compare_results_merged = compare_results_merged.transpose()
compare_results_merged

,0,1,2,3,4
docno,A,B,C,D,E
stance_label,NO,NEUTRAL,FIRST,FIRST,SECOND
docno,B,C,D,E,A
stance_label,NEUTRAL,FIRST,FIRST,SECOND,NO
docno,B,C,E,D,A
stance_label,NEUTRAL,FIRST,SECOND,FIRST,NO


In [33]:
compare_results_merged.style.to_latex("data/figure-example-ranking.tex")